# Launch with SDK

In [1]:
#Import the SdkClient from the gradient package
from gradient import sdk_client
import os
from gradient import constants
import re
from time import sleep

In [5]:
#set projec info
project = "prs3cl0zp"
cluster = "clpvt0s6x"

In [3]:
client = sdk_client.SdkClient(api_key)
experiments_list = []

## Run github repo as experiment

In [50]:
def run_single_node(project, cluster, experiments_list):
    env = {
        "EPOCHS_EVAL":"10",
        "TRAIN_EPOCHS":"40",
        "MAX_STEPS":"50000",
        "EVAL_SECS":"600",
        "apiKey_openvino": "secret:apiKey_openvino"     
    }

    single_node_parameters = { 
        "name": "single-node-sdk",
        "project_id": project,
        "container": "paperspace/openvinopipeline:mo",
        "machine_type": "P4000",
        "command": "pip install -r requirements.txt && python mnist.py",
        "experiment_env": env,
        "workspace_url": "https://github.com/Paperspace/openvino-pipeline.git", #can be local directory, a git repo or commit, or s3 bucket
        "cluster_id" : cluster,
        "model_type" : "Custom",
        "model_path" : "/artifacts"
    }
    #pass dictionary into experiments client

    experiment_id = client.experiments.run_single_node(**single_node_parameters)
    
    print(client.experiments.get(experiment_id))


    print("https://console.paperspace.com/projects/"+project+"/experiments/" + experiment_id)

    experiments_list += [experiment_id]
    
run_single_node(project, cluster, experiments_list)

In [9]:
def run_multi_node_mpi(project, cluster, experiments_list):
    multi_node_parameters = { 
        "name": "mpi-tensorflow",
        "project_id": project,
        "worker_container": "horovod/horovod:0.18.1-tf1.14.0-torch1.2.0-mxnet1.5.0-py3.6",
        "worker_machine_type": "p2.xlarge",
        "worker_command": "sleep infinity",
        "worker_count": 2,
        "master_container": "horovod/horovod:0.18.1-tf1.14.0-torch1.2.0-mxnet1.5.0-py3.6",
        "master_machine_type": "p2.xlarge",
        "master_command": "mpirun --allow-run-as-root -np 1 --hostfile /generated/hostfile  -bind-to none -map-by slot  -x NCCL_DEBUG=INFO -mca pml ob1 -mca btl ^openib python mnist_keras.py",
        "master_count": 1,
        "workspace_url": "https://github.com/Paperspace/horovod-distributed-example.git",
        "model_type": "Tensorflow",
        "cluster_id" : cluster,
    }
    experiment_id = client.experiments.run_mpi_multi_node(**multi_node_parameters)
    print("https://www.paperspace.com/console/projects/prcl68pnk/experiments/" + experiment_id)
    
    experiments_list += [experiment_id]

## State & Log Helpers 

In [52]:
def wait_for_end(experiment_id):
    print("Watching state of experiment: "+experiment_id)
    state = ""
    while state != "stopped":
         state= constants.ExperimentState.get_state_str(client.experiments.get(experiment_id).state)

In [16]:
print("adding to tensorboard")
tb_id = tb_client.create(experiments=experiments_list)

adding to tensorboard


## Wait for Experiments to end

In [13]:
for exp_id in experiments_list:
    wait_for_end(exp_id)

Watching state of experiment: escik3pmkqat4g


## View the output model

In [53]:
#get output models of all experiments
def get_experiment_models(experiment_list):
    models = []
    for exp_id in experiment_list:
        model = client.models.list(experiment_id = exp_id)
        #model was returned
        if model != None:
            print("model found for experiment "+exp_id)
            #if model summary not none
            if model[0].summary != None:
                print(model[0].summary['MSE'])
                models += [[model[0].id, model[0].summary['MSE']]]
            else:
                models += [[model[0].id, -1]]
    return models

In [54]:
#wait for model processing to finish up
models = get_experiment_models(experiments_list)
print(models)

model found for experiment e48sn35v14upc
{'max': 0.9909999966621399, 'min': 0.9671000242233276, 'var': 0.00014280217064399636, 'mean': 0.9790500104427338, 'median': 0.9790500104427338, 'stddev': 0.011949986219406128}
[['moc88lh7zsctwq', 0.9909999966621399]]


## Deploy model using SDK

In [55]:
models_sorted = sorted(models, key = lambda x : x[1], reverse=True)
[best_model, best_acc] = models_sorted[0]
print("best model is " +str(best_model) + " with acc " +str(best_acc))

best model is moc88lh7zsctwq with acc 0.9909999966621399


In [4]:
deploy_param = {
    "deployment_type" : "Custom",
    "image_url": "intelaipg/openvino-model-server",
    "command":  "/ie-serving-py/start_server.sh ie_serving model --model_path /models/model --model_name bacteria_classifier --port 9001 --rest_port 8001",
    "name": "deployment-open-vino",
    "machine_type": "C5",
    "instance_count": 1,
    "min_instance_count": 1,
    "max_instance_count": 5,
    "resource": "cpuPercentage/targetAverage:0.2",
    "cluster_id" : cluster,
    "model_id" : 'mo4uo3eaccmony',
    "ports": 8001
    "project_id": "prs3cl0zp",
    "container_model_path": "/models/model",
    "container_url_path": "/v1/models/bacteria_classifier"
}
deployment_id = client.deployments.create(**deploy_param)
client.deployments.start(deployment_id)

SyntaxError: invalid syntax (<ipython-input-4-29b1f5fea57e>, line 5)

In [ ]:
##OR update existing deployment
deploy_param["deployment_id"] = "deqj8z1ium0gi1"
client.deployments.update(**deploy_param)